# Read NCOM OoT Gulf of Mexico data
using Zarr/ReferenceFileSystem

In [ ]:
import fsspec
import xarray as xr
import json
import intake
import numpy as np
import hvplot.xarray

In [ ]:
%%time
so = {'anon':False, 'profile':'zarr-collab'}
fo = 'kyle.json'
#fo = 's3://coawst-public/rsignell/testing/ncom.total.json'
fs = fsspec.filesystem("reference", fo=fo, 
                       remote_protocol="s3", 
                       remote_options=so, skip_instance_cache=True)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr")

In [ ]:
ds

In [ ]:
ds.time

In [ ]:
ds.water_u[0,:,:].mean().values

In [ ]:
%%time
so = {'requester_pays':True}
fo = 's3://coawst-public/rsignell/testing/gom1km/gom1km.json'
#fo = 's3://coawst-public/rsignell/testing/ncom.total.json'
fs = fsspec.filesystem("reference", fo=fo, target_options=so,
                       remote_protocol="s3", target_protocol='s3',
                       remote_options=so, skip_instance_cache=True)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", decode_times=False)

In [ ]:
fs = fsspec.filesystem('s3', profile='zarr-collab')

In [ ]:
fs.ls('zarrcollab/oot/nrl/gom1km/')[:5]

In [ ]:
fs1 = fsspec.filesystem('s3', requester_pays=True)
j = fs1.open("s3://coawst-public/rsignell/testing/gom1km/gom1km.json")

In [ ]:
f = fs1.ls('s3://coawst-public/rsignell/testing/gom1km')

In [ ]:
f[1]

In [ ]:
ds = xr.open_dataset(fs1.open(f[1]))

In [ ]:
ds

In [ ]:
import json

In [ ]:
m = fsspec.get_mapper("reference://", fo=json.load(j), remote_protocol="s3", 
                              remote_options={"requester_pays": 'true'})

In [ ]:
ds = xr.open_dataset(m, engine="zarr", drop_variables='reference_time' )

In [ ]:
    m = fsspec.get_mapper("reference://", fo=json.load(afile), 
                          remote_protocol="s3", 
                          remote_options={'anon':False, 'profile':'julia'})
    ds = xr.open_dataset(m, engine="zarr", drop_variables='reference_time' )

In [ ]:
ds

In [ ]:
ds.water_u[0,:,:].mean().values

#### Load from jsons in zip into a single dataset

In [ ]:
jsons = fsspec.open_files("zip://*::s3://coawst-public/rsignell/testing/gom1km.zip",
                          s3={"requester_pays": True})
with jsons[0] as afile:
    m = fsspec.get_mapper("reference://", fo=json.load(afile), 
                          remote_protocol="s3", 
                          remote_options={'anon':False, 'profile':'julia'})
    ds = xr.open_dataset(m, engine="zarr", drop_variables='reference_time' )

In [ ]:
ds.water_u[-1,:,:].mean().values

In [ ]:
ds_list=[]
for j in jsons:
    with j as afile:
        m = fsspec.get_mapper("reference://", fo=json.load(afile), remote_protocol="s3", 
                              remote_options={"requester_pays": 'true'})
        ds_list.append(xr.open_dataset(m, engine="zarr", drop_variables='reference_time'))

ds = xr.concat(ds_list, dim='time', 
               coords='minimal', data_vars='minimal', compat='override')

In [ ]:
ds.water_u[0,:,:].mean().values

Drop the singleton depth dimension

In [ ]:
ds = ds.squeeze('depth', drop=True)

Use numpy expression for sqrt -- don't need xarray ufuncs

In [ ]:
ds['speed'] = np.sqrt(ds.water_u**2 + ds.water_v**2)

holoviz like lon to be [-180, 180]

In [ ]:
ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))

In [ ]:
ds.speed[-1,:,:].hvplot.quadmesh(x='lon', y='lat', 
                                 rasterize=True, cmap='turbo', 
                                 geo=True, tiles='OSM')

In [ ]:
cat = intake.open_catalog('s3://zarrcollab/intake_catalog.yml', 
                          storage_options={"requester_pays": True})

In [ ]:
list(cat)

In [ ]:
ds = cat['oot-nrl-socal1km3dvar'].to_dask()

In [ ]:
ds.water_u[-1,:,:].mean().values

In [ ]:
ds

#### Load multiple NetCDF files from a single json

In [ ]:
r_opts = {'anon':False, 'profile':'zarr-collab'} # NetCDF files on AWS Open Data public bucket

#t_opts = {'profile':'zarr-collab'}  # JSON or zip file on requester pays bucket
#fo = "s3://zarrcollab/oot/nrl/gom1km/gom1km.json"
#fo = 's3://zarrcollab/oot/nrl/socal1km3dvar/socal1km3dvar.json'

fo = 's3://zarrcollab/oot/nrl/gom1km/gom1km.json'
t_opts = {'anon':False, 'profile':'zarr-collab'}

fs = fsspec.filesystem("reference", fo=fo, 
                       remote_protocol='s3', remote_options=r_opts,
                       target_protocol='s3', target_options=t_opts)

m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr")

In [ ]:
ds.water_u[-1,:,:].mean().values

In [ ]:
fs.download(fo,'gom1km.json')